# CSI 4142: fundamentals of data science


ASSIGNMENT 2: DATA CLEANING

# 2. INTRODUCTION (FOR DATASET 1)



* **GOAL OF THE ANALYSIS**

The objective of this analysis is to explore and derive insights from the Netflix Movies and TV Shows dataset by examining key attributes such as genre, director, cast, release year, duration, and ratings. The goal is to identify trends, patterns, and potential correlations that can help understand content distribution, audience preferences, and overall streaming trends on Netflix.

*   **AUDIENCE**

This analysis is intended for:

* Data Analysts: To extract meaningful patterns and trends in Netflix content.
* Content Creators: To understand what types of movies and shows are popular.
* Marketing Teams: To gain insights into audience preferences and content availability.
* Streaming Industry Professionals: To compare Netflix’s content strategy with competitors.




# 2.1 DATASETS IMPORTS AND LOADING

In [ ]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher

In [ ]:

url_netflix = "https://raw.githubusercontent.com/exaucee2/CSI4142_A2/master/netflix_titles.csv"
dataset_netflix = pd.read_csv(url_netflix)

print("Dataset Netflix")
print(dataset_netflix.info())
print(dataset_netflix.head())
print(dataset_netflix.describe(include='all'))

Dataset Netflix
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB
None
  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Ju

# 2.2 MISSING VALUES

In [ ]:
def display_missing_values(df, dataset_name):
    print(f"\nOutput for {dataset_name}:")

    def display_statistics(missing, label):
        if (missing > 0).any():
            print(f"{label}:\n{missing[missing > 0]}")
        else:
            print(f"No {label.lower()} found")

    # Empty strings
    missing_empty = (df == '').sum()
    display_statistics(missing_empty, "Empty Strings")

    # '?' as a missing value
    missing_question_mark = (df == '?').sum()
    display_statistics(missing_question_mark, "Question Mark Values ('?')")

    # NaN values
    missing_NaN = df.isnull().sum()
    display_statistics(missing_NaN, "NaN Values")


datasets = [

    (dataset_netflix, 'Netflix Dataset')
]

for data, name in datasets:
    display_missing_values(data, name)


Output for Netflix Dataset:
No empty strings found
No question mark values ('?') found
NaN Values:
director      2634
cast           825
country        831
date_added      10
rating           4
duration         3
dtype: int64


# 2.3 REDUNDANCY

In [ ]:
def check_duplicate_rows(df, dataset_name):
    duplicate_rows = df[df.duplicated()]
    if not duplicate_rows.empty:
       print(f"{dataset_name}: {len(duplicate_rows)} duplicate rows found.")
       print("Duplicate rows:")
       print(duplicate_rows)
    else:
       print(f"{dataset_name}: No duplicate rows found.")
check_duplicate_rows(dataset_netflix, 'Netflix Dataset')


Netflix Dataset: No duplicate rows found.


In [ ]:
def check_duplicate_columns(df, dataset_name):
    duplicated_columns = df.T.duplicated().sum()
    if duplicated_columns > 0:
        print(f"{dataset_name}: {duplicated_columns} duplicate columns found.")
    else:
        print(f"{dataset_name}: No duplicate columns found.")

check_duplicate_columns(dataset_netflix, 'Netflix Dataset')


Netflix Dataset: No duplicate columns found.


# 3. DATASETS DESCRIPTIONS

DATASET 1:


*   **NAME**: netflix movies and tv shows
*   **AUTHOR**: not found
*   **PURPOSE**: The purpose of this dataset analysis is to:

1. Identify missing data and ensure data quality.
2. Analyze the diversity and distribution of movies and TV shows available on Netflix.
3. Explore how factors such as country, rating, and genre influence content availability.
4. Examine trends in content addition and removals over time.
5. Provide actionable insights that can be used for decision-making in content acquisition and recommendation systems.

*   **SHAPE:** this dataset contains 8807 rowa and 12 columns
*   **LIST OF FEATURES**:

| Column Name      |      Description        |         Data Type    |
|-------------------------|---------------------------|--------------|
|show_id| Unique identifier assigned to each show | Categorical (Nominal)
|type | Type of content (Movie or TV Show)| Categorical (Nominal)
|title |Title of the movie or TV showr|Categorical (Nominal)
|director | Name of the director(s)|Categorical (Nominal)
|cast|List of main actors/actresses|Categorical (Nominal)
|country |Country where the movie/TV show was produced|Categorical (Nominal)
|date_added| 	Date when the movie/TV show was added to Netflix|Categorical (Ordinal)
|release_year| Year when the movie/TV show was released|Numerical (Discrete)
|rating|Maturity rating (e.g., PG, TV-MA, etc.)|Categorical (Nominal)
|duration|	Duration of the movie (in minutes) or number of seasons for TV shows|Categorical (Nominal)
|listed_in|Categories/Genres associated with the movie/TV show|Categorical (Nominal)
|description|Brief summary of the movie/TV show|Numerical Text (Unstructured)



*   **MISSING VALUES**: Output for Netflix Dataset:
No empty strings found
No question mark values ('?') found

NaN Values:

director      2634

cast           825

country        831

date_added      10

rating           4

duration         3

*   **REDUNDANCY** :No duplicate rows or columns were found in the dataset, indicating that all features are unique and do not contain redundancy.

# 3.1 Add Errors to Each Column

In [ ]:
# Add errors to each column of the dataset

# Type error: 'release_year' should be an integer, but we put a string
dataset_netflix.loc[0, 'release_year'] = 'unknown'  # Type error

# Range error: 'duration' should be between 0 and 300 minutes, we set a negative duration
dataset_netflix.loc[10, 'duration'] = -5  # Negative duration
dataset_netflix.loc[20, 'duration'] = 500  # Duration too high

# Format error: 'date_added' should be in 'YYYY-MM-DD' format, but we set an incorrect format
dataset_netflix.loc[5, 'date_added'] = '12/15/2021'  # Wrong date format

# Consistency error: 'rating' is 'TV-MA', but 'duration' is inconsistent with this rating
dataset_netflix.loc[15, 'rating'] = 'TV-MA'  # Adult rating
dataset_netflix.loc[15, 'duration'] = 30  # Inconsistent duration for 'TV-MA'

# Presence error: 'director' should have a value, but we set it to None
dataset_netflix.loc[30, 'director'] = None  # Missing value

# Exact duplicate: Duplicate an entire row (for example, row 5)
dataset_netflix = pd.concat([dataset_netflix, dataset_netflix.iloc[5:6]])  # Duplicate row 5

# Near duplicate: Slightly different title
dataset_netflix.loc[40, 'title'] = 'The Big Bang Theory   '  # Added extra spaces
dataset_netflix.loc[41, 'title'] = 'The Big Bang Theory'  # Almost identical title

# Length error: 'title' should be longer, but we set a very short title
dataset_netflix.loc[50, 'title'] = 'A'  # Title too short

# Type error: 'type' should be a string, but we set it to an integer
dataset_netflix.loc[10, 'type'] = 12345  # Type error (expected string)

# Consistency error: 'country' and 'director' should logically relate, but we set inconsistent values
dataset_netflix.loc[100, 'country'] = 'Mars'  # Fictional country
dataset_netflix.loc[100, 'director'] = 'Unknown'  # Non-existent director

# Presence error: 'cast' should always have a value, but we set it to None
dataset_netflix.loc[200, 'cast'] = None  # Missing value

# Type error: 'show_id' should be a string, but we set it to an integer
dataset_netflix.loc[150, 'show_id'] = 987654  # Type error (expected string)

# Add a near-duplicate row with slightly different values to test near duplicates
dataset_netflix.loc[300, 'title'] = 'The Big Bang Theory'
dataset_netflix.loc[301, 'title'] = 'The Big Bang Theory '  # Added an extra space


print("Errors added successfully.")
print(dataset_netflix.head())


Errors added successfully.
  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added release_year rating   duration  \
0  September 25, 2021      unknown  PG-13     90 min   
1  September 24, 2021         2021  TV-MA  2 Seasons   
2  Se

<ipython-input-58-c708ef00f726>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unknown' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataset_netflix.loc[0, 'release_year'] = 'unknown'  # Type error


# 4. CLEAN DATA CHECKER  

# 4.1 DATA TYPE ERRORS

**Cell 1 : Test Name + Description**

In [ ]:
# Description
print("\nTest 1: Data Type Errors")
print("\nData Type Check: Verifying if each column has expected data types.")


Test 1: Data Type Errors

Data Type Check: Verifying if each column has expected data types.


**Cell 2 : Parameters for the Checker**

In [ ]:
# Define expected data types for the columns (using pandas dtypes names):
expected_dtypes = {
    'show_id': 'object',
    'type': 'object',
    'title': 'object',
    'director': 'object',
    'cast': 'object',
    'country': 'object',
    'date_added': 'object',
    'release_year': 'int64',
    'rating': 'object',
    'duration': 'object',
    'listed_in': 'object',
    'description': 'object'
}
print("Expected data types defined for columns.")

Expected data types defined for columns.


**Cell 3 : Checker Code**

In [ ]:
# Check for dtype mismatches, ensuring the column exists in expected_dtypes
dtype_errors = {
    col: dataset_netflix[col].dtype
    for col in dataset_netflix.columns
    if col in expected_dtypes and dataset_netflix[col].dtype != expected_dtypes[col]
}

print(dtype_errors)


{'release_year': dtype('O')}


**Cell 4 : Report of Findings**

In [ ]:
if dtype_errors:
    print("Data type mismatches found:")
    for col, dtype in dtype_errors.items():
        print(f"Column '{col}' has data type {dtype}, expected {expected_dtypes[col]}")
else:
    print("No data type errors found.")


Data type mismatches found:
Column 'release_year' has data type object, expected int64


# 4.2 RANGE ERRORS

**Cell 1 : Test Name + Description**

In [ ]:
# Description
print("\nTest 2: Range Errors")
print("\nRange Check: Ensuring numerical values fall within expected range.")


Test 2: Range Errors

Range Check: Ensuring numerical values fall within expected range.


**Cell 2 : Parameters for the Checker**

In [ ]:
range_checks = {
    'release_year': {'min': 1900, 'max': 2022},
    'duration': {'min': 0, 'max': 300}
}

**Cell 3 : Checker Code**

In [ ]:
range_errors = {}
for col, bounds in range_checks.items():
    if dataset_netflix[col].dtype in ['int64', 'float64']:
        out_of_range = dataset_netflix[(dataset_netflix[col] < bounds['min']) | (dataset_netflix[col] > bounds['max'])]
        range_errors[col] = out_of_range

**Cell 4 : Report of Findings**

In [ ]:
print("Range Errors Dictionary:")
print(range_errors)
for col, errors in range_errors.items():
    if not errors.empty:
        print(f"Range errors in column '{col}':")
        print(errors[['title', col]])
        print()
    else:
        print(f"No range errors in column '{col}'.")



Range Errors Dictionary:
{}


# 4.3 FORMAT ERRORS

**Cell 1 : Test Name + Description**

In [ ]:
 #Description
print("\nTest 3: Format Errors")
print("\nFormat Check: Ensuring correct formatting of values.")


Test 3: Format Errors

Format Check: Ensuring correct formatting of values.


**Cell 2 : Parameters for the Checker**

In [ ]:
format_rules = {
    'date_added': r'\d{2}/\d{2}/\d{4}'  # Example: MM/DD/YYYY
}




**Cell 3 : Checker Code**

In [ ]:

format_errors = {}
for col, pattern in format_rules.items():
    invalid_rows = dataset_netflix[~dataset_netflix[col].astype(str).str.match(pattern, na=False)]
    format_errors[col] = invalid_rows

**Cell 4 : Report of Findings**

In [ ]:
for col, errors in format_errors.items():
    print(f"Column '{col}' has {len(errors)} format errors.")
    if not errors.empty:
        print(errors)

Column 'date_added' has 8806 format errors.
     show_id     type                  title         director  \
0         s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1         s2  TV Show          Blood & Water              NaN   
2         s3  TV Show              Ganglands  Julien Leclercq   
3         s4  TV Show  Jailbirds New Orleans              NaN   
4         s5  TV Show           Kota Factory              NaN   
...      ...      ...                    ...              ...   
8802   s8803    Movie                 Zodiac    David Fincher   
8803   s8804  TV Show            Zombie Dumb              NaN   
8804   s8805    Movie             Zombieland  Ruben Fleischer   
8805   s8806    Movie                   Zoom     Peter Hewitt   
8806   s8807    Movie                 Zubaan      Mozez Singh   

                                                   cast        country  \
0                                                   NaN  United States   
1     Ama Qamata, Khosi Nge

# 4.4 CONSISTENCY ERRORS

**Cell 1 : Test Name + Description**

In [ ]:
 #Description
print("\nTest 4: CONSISTENCY ERRORS")
print("\nConsistency Check: Ensuring logical consistency across columns.")



Test 4: CONSISTENCY ERRORS

Consistency Check: Ensuring logical consistency across columns.


**Cell 2 : Parameters for the Checker**

In [ ]:
consistency_condition = (dataset_netflix['type'] == 'Movie') & (dataset_netflix['duration'].str.contains('Season', na=False))

**Cell 3 : Checker Code**

In [ ]:
consistency_errors = dataset_netflix[consistency_condition]

**Cell 4 : Report of Findings**

In [ ]:
print(f"{len(consistency_errors)} inconsistencies found where 'Movie' has 'Seasons' in duration.")
if not consistency_errors.empty:
    print(consistency_errors[['title', 'type', 'duration']])

0 inconsistencies found where 'Movie' has 'Seasons' in duration.


# 4.5 UNIQUENESS ERROS

**Cell 1 : Test Name + Description**

In [ ]:
print("\nTest 5: uniqueness errors")
print("\nUniqueness Check: Identifying duplicate titles.")


Test 5: uniqueness errors

Uniqueness Check: Identifying duplicate titles.


**Cell 2 : Parameters for the Checker**

In [ ]:
# Specify the attribute that should have unique values:
test_attribute = 'title'
print("Parameter defined: 'title' should be unique.")

Parameter defined: 'title' should be unique.


**Cell 3 : Checker Code**

In [ ]:
unique_errors = dataset_netflix[dataset_netflix.duplicated(subset=[test_attribute], keep=False)]

**Cell 4 : Report of Findings**

In [ ]:
print(f"{len(unique_errors)} duplicate titles found.")
if not unique_errors.empty:
    print(unique_errors[['title', 'type']])

4 duplicate titles found.
                   title     type
5          Midnight Mass  TV Show
41   The Big Bang Theory    Movie
300  The Big Bang Theory    Movie
5          Midnight Mass  TV Show


# 4.6 PRESENCE ERRORS

**Cell 1 : Test Name + Description**

In [ ]:
print("Test 6: Presence Errors")
print("\nPresence Check: Detecting missing values.")

Test 6: Presence Errors

Presence Check: Detecting missing values.


**Cell 2 : Parameters for the Checker**

In [ ]:
print("We will check all columns for missing (null) values")

We will check all columns for missing (null) values


**Cell 3 : Checker Code**

In [ ]:
missing_values = dataset_netflix.isnull().sum()

**Cell 4 : Report of Findings**

In [ ]:
print("Columns with missing values:")
print(missing_values[missing_values > 0])

Columns with missing values:
director      2634
cast           826
country        831
date_added      10
rating           4
duration         3
dtype: int64


# 4.7 LENGTH ERRORS

**Cell 1 : Test Name + Description**

In [ ]:
print("Test 7: Length Errors")
print("\nLength Check: Ensuring appropriate title lengths.")

Test 7: Length Errors

Length Check: Ensuring appropriate title lengths.


**Cell 2 : Parameters for the Checker**

In [ ]:
test_attribute = 'title'
min_length = 3


**Cell 3 : Checker Code**

In [ ]:
dataset_netflix['title_length'] = dataset_netflix[test_attribute].str.len()
short_titles = dataset_netflix[dataset_netflix['title_length'] < min_length]

**Cell 4 : Report of Findings**

In [ ]:
print(f"{len(short_titles)} titles are too short.")
if not short_titles.empty:
    print(short_titles[['title']])

13 titles are too short.
     title
50       A
2069     H
2127    3%
3668    4L
4090    Z4
4177    IO
4660    PK
5388    46
5958     9
5959    21
7155     K
7687     P
8765    XX


# 4.8 LOOK-UP ERRORS

**Cell 1 : Test Name + Description**

In [ ]:
print("Test 8: Look-up Errors")
print("\nLook-up Check: Validating ratings against predefined categories.")

Test 8: Look-up Errors

Look-up Check: Validating ratings against predefined categories.


**Cell 2 : Parameters for the Checker**

In [ ]:
test_attribute = 'rating'
valid_ratings = ['G', 'PG', 'PG-13', 'R', 'TV-MA', 'TV-14']

**Cell 3 : Checker Code**

In [ ]:
lookup_errors = dataset_netflix[~dataset_netflix[test_attribute].isin(valid_ratings)]

**Cell 4 : Report of Findings**

In [ ]:
print(f"{len(lookup_errors)} invalid ratings found.")
if not lookup_errors.empty:
    print(lookup_errors[['title', 'rating']])

1823 invalid ratings found.
                                             title rating
13                Confessions of an Invisible Girl  TV-PG
22                                 Avvai Shanmughi  TV-PG
23    Go! Go! Cory Carson: Chrissy Takes the Wheel   TV-Y
26                                  Minsara Kanavu  TV-PG
34                         Tayo and Little Wizards  TV-Y7
...                                            ...    ...
8795                               Yu-Gi-Oh! Arc-V  TV-Y7
8796                                    Yunus Emre  TV-PG
8797                                     Zak Storm  TV-Y7
8800                            Zindagi Gulzar Hai  TV-PG
8803                                   Zombie Dumb  TV-Y7

[1823 rows x 2 columns]


# 4.9 EXACT DUPLICATE ERRORS

**Cell 1 : Test Name + Description**

In [ ]:

print("Test 9: Exact Duplicate Errors")
print("\nExact Duplicate Check: Identifying identical rows.")

Test 9: Exact Duplicate Errors

Exact Duplicate Check: Identifying identical rows.


**Cell 2 : Parameters for the Checker**

In [ ]:
print("we are checking for exact duplicates across all columns in the dataset. ")

we are checking for exact duplicates across all columns in the dataset. 


**Cell 3 : Checker Code**

In [ ]:
exact_duplicates = dataset_netflix[dataset_netflix.duplicated()]

**Cell 4 : Report of Findings**

In [ ]:
print(f"{len(exact_duplicates)} exact duplicate rows found.")
if not exact_duplicates.empty:
    print(exact_duplicates)

1 exact duplicate rows found.
  show_id     type          title       director  \
5      s6  TV Show  Midnight Mass  Mike Flanagan   

                                                cast country  date_added  \
5  Kate Siegel, Zach Gilford, Hamish Linklater, H...     NaN  12/15/2021   

  release_year rating  duration                           listed_in  \
5         2021  TV-MA  1 Season  TV Dramas, TV Horror, TV Mysteries   

                                         description  title_length  
5  The arrival of a charismatic young priest brin...            13  


# 4.10 NEAR DUPLICATE ERRORS

**Cell 1 : Test Name + Description**

In [ ]:
print("Test 10: Near Duplicate Errors")
print("\nNear Duplicate Check: Finding similar records.")


Test 10: Near Duplicate Errors

Near Duplicate Check: Finding similar records.


**Cell 2 : Parameters for the Checker**

In [ ]:
similarity_threshold = 0.9

**Cell 3 : Checker Code**

In [ ]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

near_duplicates = []
for i in range(len(dataset_netflix) - 1):
    if similar(dataset_netflix.iloc[i]['title'], dataset_netflix.iloc[i + 1]['title']) > similarity_threshold:
        near_duplicates.append((dataset_netflix.iloc[i], dataset_netflix.iloc[i + 1]))

**Cell 4 : Report of Findings**

In [ ]:
if near_duplicates:
    for i, (record1, record2) in enumerate(near_duplicates):
        print(f"Near-Duplicate Pair {i + 1}:")
        print(f"Title 1: {record1['title']}")
        print(f"Title 2: {record2['title']}")
        print("---")

Near-Duplicate Pair 1:
Title 1: The Big Bang Theory   
Title 2: The Big Bang Theory
---
Near-Duplicate Pair 2:
Title 1: House Party
Title 2: House Party 2
---
Near-Duplicate Pair 3:
Title 1: House Party 2
Title 2: House Party 3
---
Near-Duplicate Pair 4:
Title 1: Kyaa Kool Hai Hum
Title 2: Kyaa Kool Hain Hum 3
---
Near-Duplicate Pair 5:
Title 1: Ragini MMS
Title 2: Ragini MMS 2
---
Near-Duplicate Pair 6:
Title 1: The Big Bang Theory
Title 2: The Big Bang Theory 
---
Near-Duplicate Pair 7:
Title 1: The Twilight Saga: Breaking Dawn: Part 1
Title 2: The Twilight Saga: Breaking Dawn: Part 2
---
Near-Duplicate Pair 8:
Title 1: Kung Fu Panda
Title 2: Kung Fu Panda 2
---
Near-Duplicate Pair 9:
Title 1: The Karate Kid Part II
Title 2: The Karate Kid Part III
---
Near-Duplicate Pair 10:
Title 1: Osuofia in London
Title 2: Osuofia in London II
---
Near-Duplicate Pair 11:
Title 1: Seven Souls in the Skull Castle: Season Moon Jogen
Title 2: Seven Souls in the Skull Castle: Season Moon Kagen
---
Ne

In [ ]:
# Save the altered dataset for validation
dataset_netflix.to_csv('altered_netflix_data.csv', index=False)
from google.colab import files
files.download('altered_netflix_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 5. CONCLUSION

The data cleaning process for the Netflix dataset highlighted several critical issues, including data type mismatches, range errors, formatting inconsistencies, missing values, and duplicate entries. These issues can compromise the accuracy and reliability of any analysis, leading to biased or incorrect insights. By addressing these errors, such as correcting data types, handling missing values, and removing duplicates, we ensure that the dataset is in optimal shape for further analysis. A clean dataset is essential for reliable decision-making and predictive modeling. This analysis demonstrates the importance of thorough data preparation to ensure that subsequent results are both accurate and actionable. Going forward, the modified dataset is ready for use in more accurate and meaningful analyses. Proper data cleaning not only improves model performance but also enhances the overall credibility of the data-driven insights.

# 6. REFERENCES

- pandas documentation — pandas 2.2.3 documentation. (s. d.). https://pandas.pydata.org/pandas-docs/stable/


- Icodewithben. (2024, 15 novembre). Data Validation in Python : Range, Type, Presence and Format Check. Medium.
https://medium.com/@icodewithben/data-validation-in-python-range-type-presence-and-form-aaefe8835a86


- GeeksforGeeks. (2024, 2 décembre). Working with Missing Data in Pandas. GeeksforGeeks. https://www.geeksforgeeks.org/working-with-missing-data-in-pandas/

- How to validate numeric ranges in Python | LabEx. (s. d.). LabEx. https://labex.io/tutorials/python-how-to-validate-numeric-ranges-in-python-422109

